In [1]:
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tqdm_notebook as tqdm
from keras.utils.data_utils import Sequence
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
from keras.models import Sequential, Model
from keras.layers import Activation, Dense, Dropout, Flatten, Convolution2D, GlobalAveragePooling2D, GlobalMaxPooling2D, MaxPooling2D
from keras import applications
from keras import optimizers

DATA_DIR = "../data/"
NUM_CLASSES = 228

C:\Users\matt\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
with open(DATA_DIR + "test.json") as test:
    test_json = json.load(test)
    
# test_urls = [obj['url'] for obj in test_json['images']]
test_paths = ["../data/test/{}.jpg".format(obj['imageId']) for obj in test_json['images']]
print(test_paths[:3])

['../data/test/1.jpg', '../data/test/2.jpg', '../data/test/3.jpg']


In [3]:
# IMAGE_SIZE = 100
# conv_base = applications.Xception(weights = "imagenet", include_top=False, input_shape = (IMAGE_SIZE, IMAGE_SIZE, 3))
# for layer in conv_base.layers[:3]:
#     layer.trainable = False
# model = Sequential()
# model.add(conv_base)
# model.add(Flatten())
# model.add(Dense(1024, activation='relu'))
# model.add(Dropout(0.4))
# model.add(Dense(1024, activation='relu'))
# model.add(Dense(NUM_CLASSES, activation='softmax'))
# model.load_weights(DATA_DIR + "model.best.100.hdf5")

# model.compile(
#     loss = "categorical_crossentropy", 
#     optimizer = optimizers.SGD(lr=0.0, momentum=0.9, decay=0.0, nesterov=False),
#     metrics=["accuracy"]
# )

IMAGE_SIZE = 256
conv_base = applications.Xception(weights = "imagenet", include_top=False, input_shape = (IMAGE_SIZE, IMAGE_SIZE, 3))

for layer in conv_base.layers[:3]:
    layer.trainable = False

model = Sequential()
model.add(conv_base)
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(1024, activation='relu'))
model.add(Dense(NUM_CLASSES, activation='softmax'))
model.load_weights(DATA_DIR + "model.xcept.256.hdf5")
model.compile(
    loss = "categorical_crossentropy", 
    optimizer = optimizers.SGD(lr=0.01, momentum=0.9), 
    # optimizer = optimizers.SGD(lr=0.0, momentum=0.9, decay=0.0, nesterov=False),
    metrics=["accuracy"]
)

In [4]:
class TestBatchSequence(Sequence):
    def __init__(self, x_set, batch_size, resize = False):
        self.x = x_set
        self.batch_size = batch_size
        self.resize = resize

    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        images = np.empty([len(batch_x), IMAGE_SIZE, IMAGE_SIZE, 3])
        for i, path in enumerate(batch_x):
            try:
                if self.resize:
                    img = Image.open(path)
                    img.thumbnail((IMAGE_SIZE, IMAGE_SIZE))
                    image = np.array(img)
                else:
                    image = np.array(Image.open(path))
            except Exception as e:
                print(e)
                output = [1]*(IMAGE_SIZE*IMAGE_SIZE*3)
                output = np.array(output).reshape(IMAGE_SIZE,IMAGE_SIZE,3).astype('uint8')
                image = Image.fromarray(output).convert('RGB')
            images[i, ...] = image
        return images 

In [6]:
%%time
BATCH = 64
STEPS = len(test_paths) // BATCH

# test_seq = TestBatchSequence(test_paths, BATCH, resize = True)
test_seq = TestBatchSequence(test_paths, BATCH, resize = False)

probs = model.predict_generator(
    test_seq,
    steps = STEPS + 1,
    workers = 5,
    verbose = 1
)

621/621 [==============================] - ETA: 5: - ETA: 5: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3:

In [7]:
probs[0]

array([9.54554935e-07, 3.03342147e-03, 7.86876917e-05, 2.59941327e-04,
       2.38500186e-04, 2.45332543e-04, 3.80298519e-03, 6.70182999e-05,
       8.43210437e-04, 9.98339310e-05, 3.41876759e-04, 2.61510140e-05,
       5.08958357e-04, 9.29917616e-04, 1.03101972e-03, 6.50179345e-07,
       5.79797141e-02, 7.60151260e-03, 2.52699610e-02, 2.21377555e-02,
       1.20388031e-04, 2.40078334e-05, 3.49973106e-06, 3.26875374e-06,
       5.05328586e-04, 8.35255429e-04, 1.50950027e-05, 1.19565765e-03,
       1.00498983e-05, 2.36800843e-04, 1.14327238e-04, 1.29744841e-03,
       1.55035945e-04, 1.14679184e-04, 1.81504729e-04, 1.18211051e-02,
       7.27815146e-04, 6.88128930e-04, 2.05254721e-04, 4.08001215e-04,
       9.92947434e-07, 2.22320508e-04, 1.54077570e-04, 1.85071621e-02,
       8.29508936e-05, 1.22713823e-06, 9.92098940e-04, 6.40111801e-04,
       1.75127145e-02, 1.38331134e-05, 7.10855646e-04, 6.17460173e-04,
       2.33984906e-02, 8.55926846e-05, 7.09463900e-04, 5.11310122e-04,
      

In [11]:
def generate_prob_labels(probas):
    label_preds = []
    for i in range(len(probas)):
        labels = []
        proba = list(probas[i])
        for i, elem in enumerate(proba):
            if elem > 0.04:
                labels.append(i + 1)
        label_preds.append(labels)
    return label_preds

probas = generate_prob_labels(probs)
print(probas[:5])

[[17, 66, 105, 153, 171, 214], [17, 105, 106, 153, 171, 214, 222], [17, 105, 153, 171, 214, 222], [17, 105, 153, 171, 214, 222], [17, 66, 105, 153, 171, 214]]


In [12]:
with open("submission-new.csv","w") as f:
    f.write("image_id,label_id\n")
    for i, labels in tqdm(enumerate(probas), total = len(probas)):
        output_labels = " ".join(str(x) for x in labels)
        f.write("{},{}\n".format(i + 1, output_labels))

HBox(children=(IntProgress(value=0, max=39706), HTML(value='')))